# Fine-tuning the YOLO model from Recorded Images

# Requirements

In [1]:
!pip install roboflow
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 135.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.4 MB/s eta 0:00:00


# Download dataset from Roboflow

In [8]:
import os
import pathlib
import shutil
import torch
from ultralytics import YOLO
from roboflow import Roboflow
from dotenv import load_dotenv

load_dotenv()

current_dir = pathlib.Path.cwd()
ROOT_DIR = current_dir

DATASET_DIR = ROOT_DIR / "data" / "roboflow_chess_dataset"
EPOCHS = 100


def download_dataset():
    ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace("test-wzcdu").project("chess-pieces-detection-camera-pxzh6")
    version = project.version(4) # No augmentation
    dataset = version.download("yolov8")

    print("Original Dataset location: ", dataset.location)
    print("Moving dataset to: ", DATASET_DIR)
    shutil.move(dataset.location, str(DATASET_DIR))


def train_yolo_model():

    # Check if GPU/MPS is available
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    # Load a pre-trained YOLO model (you can choose different sizes: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x)
    model = YOLO("yolov8s.pt")

    print("Using device: ", device)
    print("Dataset file: ", os.path.join(str(DATASET_DIR), "data.yaml"))
    print("Project: ", str(ROOT_DIR / "models" / "yolo" / "finetuned"))

    # Train the model on the custom dataset
    model.train(
        data=os.path.join(str(DATASET_DIR), "data.yaml"),
        epochs=EPOCHS,
        imgsz=1024,  # Multiple of 32 (1216, 736)
        batch=15,
        device=device,
        project=str(ROOT_DIR / "models" / "yolo" / "finetuned"),
        name="chess_yolov8s_finetuned",
        exist_ok=True,
        #augment=False, # Dataset already has augmentation
        # plots=False,
        # cache=False,
        # workers=0,
        # val=False,  # Skip validation initially
        #degrees=90.0,      # rotate images randomly between -90° and +90°
        #fliplr=0.5,        # horizontal flip with 50% probability
        #flipud=0.5,        # vertical flip with 50% probability
        #scale=0.5,         # random scaling (zoom in/out)
        #shear=10.0,        # shear images up to 10 degrees
        #perspective=0.001, # small perspective warping
        #hsv_h=0.015,       # color hue augmentation
        #hsv_s=0.7,         # color saturation
        #hsv_v=0.4,         # color value (brightness)

        # --- Refined augmentation parameters ---
        degrees=90.0,       # keep strong rotation (critical for camera angle variation)
        fliplr=0.5,         # horizontal flip (still valuable)
        flipud=0.5,         # vertical flip (also important)

        # Reduced geometric distortion
        scale=0.1,          # only slight zoom in/out (was 0.5)
    )

    # Evaluate the model
    results = model.val()
    print(results)


def train_with_cli():
    import subprocess

    cmd = [
        "yolo",
        "train",
        f"data={DATASET_DIR}/data.yaml",
        "model=yolov8s.pt",
        f"epochs={EPOCHS}",
        "imgsz=1216",
        "batch=10",
        f"project={ROOT_DIR}/models/yolo/finetuned",
        "name=chess_yolov8m_finetuned",
        "exist_ok=True",
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    return result.returncode == 0

In [3]:
download_dataset()

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to chess-pieces-detection-camera-4 in yolov8:: 100%|██████████| 812/812 [00:00<00:00, 6671.33it/s]


Original Dataset location:  /content/chess-pieces-detection-camera-4
Moving dataset to:  /content/data/roboflow_chess_dataset


In [ ]:
!cat .env

ROBOFLOW_API_KEY=T7c5mO9VBnqQWGU5EO0L

In [9]:
train_yolo_model()

Using device:  cuda
Dataset file:  /content/data/roboflow_chess_dataset/data.yaml
Project:  /content/models/yolo/finetuned
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=15, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/roboflow_chess_dataset/data.yaml, degrees=90.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, nam

In [ ]:
train_with_cli()

STDOUT: 
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=10, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/roboflow_chess_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1216, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=chess_yolov8m_finetuned, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, p

False

In [ ]:
def train_with_cli():
    import subprocess
    import os

    cmd = [
        "yolo",
        "train",
        f"data={DATASET_DIR}/data.yaml",
        "model=yolov8m.pt",
        f"epochs={EPOCHS}",
        "imgsz=1216",
        "batch=10",
        f"project={ROOT_DIR}/models/yolo/finetuned",
        "name=chess_yolov8m_finetuned",
        "exist_ok=True",
    ]

    try:
        # Use subprocess.Popen to run the command and stream the output
        # Setting universal_newlines=True ensures the output is text
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Loop through the output line by line and print it
        for line in process.stdout:
            print(line, end='')

        # Wait for the process to complete and get the return code
        process.wait()

        # Check the return code to see if the process was successful
        if process.returncode == 0:
            print("\nYOLO training completed successfully!")
            return True
        else:
            print(f"\nYOLO training failed with return code: {process.returncode}")
            return False

    except FileNotFoundError:
        print("Error: 'yolo' command not found. Make sure Ultralytics YOLO is installed and in your PATH.")
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

train_with_cli()


Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=10, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/roboflow_chess_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1216, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=chess_yolov8m_finetuned, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10

True

In [ ]:
!ls

sample_data  yolo11n.pt  yolov8s.pt


# Zip to download

In [10]:
!zip -r models_aug_3.zip models/

  adding: models/ (stored 0%)
  adding: models/yolo/ (stored 0%)
  adding: models/yolo/finetuned/ (stored 0%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/ (stored 0%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/BoxF1_curve.png (deflated 8%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/args.yaml (deflated 53%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/labels.jpg (deflated 31%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/val_batch0_labels.jpg (deflated 3%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/train_batch2161.jpg (deflated 10%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/train_batch0.jpg (deflated 4%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/train_batch1.jpg (deflated 3%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/train_batch2160.jpg (deflated 10%)
  adding: models/yolo/finetuned/chess_yolov8s_finetuned/train_batch2.jpg (deflated 4%)
  adding: models/yolo/finetuned/chess

In [ ]:
!unzip models.zip -d models/

Archive:  models.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of models.zip or
        models.zip.zip, and cannot find models.zip.ZIP, period.


# Fine-tuning cli

In [ ]:
!yolo task=detect mode=train model=yolov8m data={dataset.location}/data.yaml epochs=100 imgsz=1000